In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import numpy as np
from aifeel.util.evaluate import evaluate_model
import dill

# Evaluate Each Model

In [ ]:
# Load the model from the file
with open("export/model/NNClassifier/model.dill", "rb") as f:
    loaded_model = dill.load(f)

with open("export/model/NNClassifier/vectorizer.dill", "rb") as f:
    vectorizer = dill.load(f)

d:\Python310\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
data = np.load('export/model/NNClassifier/data.npz')

# Access the arrays stored in the .npz file
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

